# Preprocessing .mat for NWB

In [ ]:
import numpy as np
import h5py
import os
import yaml
import pandas as pd
from datetime import datetime
import shutil
import importlib
import numpy as np
import pandas as pd
import convert_to_nwb_for_PB
import converters.Initiation_nwb
import scipy.io
import warnings
from datetime import timedelta
from tqdm import tqdm
import time
import shutil



# Load data 

## .mat files

In [37]:

mat_file = "/Users/lorisfabbro/Documents/MATLAB/PB/mPFC_Preprocessed.mat"
output_folder = "data"

### General

INFO :
- Acceder keys h5py.file :  list(group.keys())
- Lire dataset [()] : roup = f['Data_Full']


In [55]:
#choice_mouses = [ "LB010", "LB011", "LB012", "LB013", "LB014", "LB015", "LB017", "LB018", "LB019", "LB021", "LB022", "LB023", "LB024", "LB025", "LB026", "LB027", "LB028", "LB030", "LB031", "LB032", "LB033", "LB034", "LB035", "LB036", "LB037", "LB039", "LB042", "LB043", "LB044", "LB045", "LB046", "LB047", "LS001", "LS003"]#, "SC014", "SC016", "SC017"] # All without SC
#choice_mouses = [ "LB010",  "LB012", "LB013",  "LB015", "LB017","LB019",  "LB022", "LB023",  "LB025",  "LB027", "LB028",  "LB031", "LB032",  "LB034",  "LB036", "LB037", "LB039", "LB042", "LB043",  "LB045", "LB047", "LS001", "LS003"] #Semi-all 
#choice_mouses = ["LB010", "LB011", 'LB012', 'LB039', 'LB042', 'LS001']
choice_mouses = ["LB010"]
mat_file = mat_file
#--------
importlib.reload(converters.Initiation_nwb)
csv_data = converters.Initiation_nwb.files_to_dataframe(mat_file = mat_file, choice_mouses = choice_mouses)

 33%|███▎      | 1/3 [00:01<00:02,  1.01s/it]

Checkpoint 0: File opened successfully


 67%|██████▋   | 2/3 [00:02<00:01,  1.13s/it]

Checkpoint 1: Mouse metadata loaded


100%|██████████| 3/3 [00:56<00:00, 25.18s/it]

...

4it [00:57, 15.76s/it]                       

Checkpoint 2.2: Sweep behavior data loaded


5it [00:58, 11.68s/it]


Starting filling the dataframe to begin the NWB conversion


Mouses Cells: 100%|██████████| 66/66 [00:00<00:00, 188.13it/s]


In [44]:
display(csv_data.columns)
display(csv_data[['Mouse Name', 'User (user_userName)', 'Cell_ID', 'Ear tag',
        'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
        'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
        'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
        'Session Type', 'Opto Session', 'Mouse Age (d)', 'Weight of Reference',
        'Weight Session']].head(5))
display(csv_data.iloc[0]["sweeps"][:1])

Index(['Mouse Name', 'User (user_userName)', 'Cell_ID', 'Ear tag',
       'Start date (dd.mm.yy)', 'End date', 'Sex_bin', 'strain', 'mutations',
       'Birth date', 'licence', 'DG', 'ExpEnd', 'Created on', 'Session',
       'Session Date (yyymmdd)', 'Start Time (hhmmss)', 'Behavior Type',
       'Session Type', 'sweeps', 'Opto Session', 'Mouse Age (d)',
       'Weight of Reference', 'Weight Session'],
      dtype='object')

,Mouse Name,User (user_userName),Cell_ID,Ear tag,Start date (dd.mm.yy),End date,Sex_bin,strain,mutations,Birth date,...,Created on,Session,Session Date (yyymmdd),Start Time (hhmmss),Behavior Type,Session Type,Opto Session,Mouse Age (d),Weight of Reference,Weight Session
0,LB010,LB,LB010_S1_R1,Unknown,08.05.18,08.05.18,M,C57BL/6JRj,WT,25.02.2018,...,Unknown,LB010_S1_R1_20180508,20180508,134938,Free Whisking & Whisker Stim,Naive,NaN,Unknown,Unknown,Unknown
1,LB010,LB,LB010_S1_R3,Unknown,08.05.18,08.05.18,M,C57BL/6JRj,WT,25.02.2018,...,Unknown,LB010_S1_R3_20180508,20180508,152232,Free Whisking & Whisker Stim,Naive,NaN,Unknown,Unknown,Unknown


[{'Sweep Index': 0,
  'Sweep Start Time': datetime.datetime(2018, 5, 8, 13, 49, 38),
  'Sweep Type': 'Free Whisking',
  'behav': array([[ 1.,  0.,  0.,  0.,  2.],
         [ 5.,  0.,  0.,  0.,  2.],
         [ 9.,  0.,  0.,  0.,  2.],
         [13.,  0.,  0.,  0.,  2.],
         [17.,  0.,  0.,  0.,  2.],
         [21.,  0.,  0.,  0.,  2.],
         [25.,  0.,  0.,  0.,  2.],
         [29.,  0.,  0.,  0.,  2.]]),
  'membrane_potential': {'sampling_rate_Hz': 20000,
   'data': [-0.05943020914919996,
    -0.05943349838796658,
    -0.05942691991043379,
    -0.059423630671668067,
    -0.05941376295537355,
    -0.05940389523908307,
    -0.05943020914919996,
    -0.05941047371660962,
    -0.05941047371660962,
    -0.059446655343037495,
    -0.05946968001442873,
    -0.059476258491973105,
    -0.05942691991043379,
    -0.05943678762673364,
    -0.059463101536886165,
    -0.059463101536886165,
    -0.059502572402168386,
    -0.059509150879721646,
    -0.05951901859605489,
    -0.059512440118498

In [45]:
print(csv_data["Session Type"].unique())
print(csv_data["Behavior Type"].unique())

['Naive']
['Free Whisking & Whisker Stim']


# TEST

In [46]:
csv_data_row = csv_data.iloc[0]
csv_data_row

Mouse Name                                                            LB010
User (user_userName)                                                     LB
Cell_ID                                                         LB010_S1_R1
Ear tag                                                             Unknown
Start date (dd.mm.yy)                                              08.05.18
End date                                                           08.05.18
Sex_bin                                                                   M
strain                                                           C57BL/6JRj
mutations                                                                WT
Birth date                                                       25.02.2018
licence                                                                1628
DG                                                                      NaN
ExpEnd                                                                  NaN
Created on  

In [54]:
sweep_data = csv_data_row["sweeps"]
trial_onsets = np.array([])
stim_indices = np.array([])
stim_amps = np.array([])
response_data = np.array([])
Sweep_IDs = np.array([])
Sweep_Start_time = np.array([])
sweep_type = np.array([])
for One_sweep in sweep_data:
    print(type(One_sweep))
    print(One_sweep.keys())
    for One_trial in One_sweep["trials"]:
        print(type(One_trial))
        print(One_trial.keys())
        trial_onsets = np.append(trial_onsets, One_trial["time_abs"].second)
        stim_indices = np.append(stim_indices, One_trial["has_stim"])
        stim_amps = np.append(stim_amps, One_trial["amplitude"])
        response_data = np.append(response_data, One_trial["response"])
        Sweep_IDs = np.append(Sweep_IDs, One_sweep["Sweep Index"])
        Sweep_Start_time = np.append(Sweep_Start_time, One_sweep["Sweep Start Time"].second)
        sweep_type = np.append(sweep_type, One_sweep["Sweep Type"])

trial_onsets = trial_onsets.astype(np.float64)
stim_indices = stim_indices.astype(np.int64)
stim_amps = stim_amps.astype(np.int64)

response_labels = np.full(response_data.shape, '', dtype=object)
response_labels[response_data == 1] = "hit"
response_labels[response_data == 0] = "miss"
response_labels[response_data == 2] = "CR"
response_labels[response_data == 3] = "FA"

Sweep_IDs = Sweep_IDs.astype(np.int64)
Sweep_Start_time = Sweep_Start_time.astype(np.float64)
sweep_type = sweep_type.astype(str)

print(trial_onsets[:30],trial_onsets.shape)
print(stim_indices[:30],stim_indices.shape)
print(stim_amps[:30],stim_amps.shape,np.unique(stim_amps))
print(response_labels[:30],response_labels.shape, np.unique(response_labels))
print(Sweep_IDs[:30],Sweep_IDs.shape, np.unique(Sweep_IDs))
print(Sweep_Start_time[:30],Sweep_Start_time.shape)
print(sweep_type[:30],sweep_type.shape, np.unique(sweep_type))

<class 'dict'>
dict_keys(['Sweep Index', 'Sweep Start Time', 'Sweep Type', 'behav', 'membrane_potential', 'current_monitor', 'whisker_angle', 'ap_times', 'reward_times', 'trials'])
<class 'dict'>
dict_keys(['trial_index', 'time_rel_s', 'time_abs', 'lick', 'reward', 'response', 'has_stim', 'amplitude', 'duration_s', 'type'])
<class 'dict'>
dict_keys(['trial_index', 'time_rel_s', 'time_abs', 'lick', 'reward', 'response', 'has_stim', 'amplitude', 'duration_s', 'type'])
<class 'dict'>
dict_keys(['trial_index', 'time_rel_s', 'time_abs', 'lick', 'reward', 'response', 'has_stim', 'amplitude', 'duration_s', 'type'])
<class 'dict'>
dict_keys(['trial_index', 'time_rel_s', 'time_abs', 'lick', 'reward', 'response', 'has_stim', 'amplitude', 'duration_s', 'type'])
<class 'dict'>
dict_keys(['trial_index', 'time_rel_s', 'time_abs', 'lick', 'reward', 'response', 'has_stim', 'amplitude', 'duration_s', 'type'])
<class 'dict'>
dict_keys(['trial_index', 'time_rel_s', 'time_abs', 'lick', 'reward', 'response

In [ ]:
sweep_data = csv_data_row["sweeps"]
trial_onsets = []
i =0 
for One_sweep in sweep_data:
    for One_trial in One_sweep["trials"]:
        i = i+1
        print(i)
        trial_onsets.append(One_trial["time_abs"])


In [ ]:
sweep_data

 # TOTAL _-_-_-_-_-_-_-_
 

In [40]:
serveur = "/Volumes/Petersen-Lab/z_LSENS/Share/Loris Fabbro/LB/NWB_files/"
local = "data/"
importlib.reload(convert_to_nwb_for_PB)
nwb_path = convert_to_nwb_for_PB.convert_data_to_nwb_PB(mat_file=mat_file, output_folder=local, mouses_name = ["LB010"])

**************************************************************************
-_-_-_-_-_-_-_-_-_-_-_-_-_-_- NWB conversion _-_-_-_-_-_-_-_-_-_-_-_-_-_-_
📥 Collecting data from .mat file: /Users/lorisfabbro/Documents/MATLAB/PB/mPFC_Preprocessed.mat


 33%|███▎      | 1/3 [00:01<00:02,  1.01s/it]

Checkpoint 0: File opened successfully


 67%|██████▋   | 2/3 [00:02<00:01,  1.13s/it]

Checkpoint 1: Mouse metadata loaded


...

5it [00:58, 11.80s/it]


Starting filling the dataframe to begin the NWB conversion


Mouses Cells: 100%|██████████| 66/66 [00:00<00:00, 126.85it/s]


,Mouse Name,User (user_userName),Cell_ID,Ear tag,Start date (dd.mm.yy),End date,Sex_bin,strain,mutations,Birth date,...,Created on,Session,Session Date (yyymmdd),Start Time (hhmmss),Behavior Type,Session Type,Opto Session,Mouse Age (d),Weight of Reference,Weight Session
0,LB010,LB,LB010_S1_R1,Unknown,08.05.18,08.05.18,M,C57BL/6JRj,WT,25.02.2018,...,Unknown,LB010_S1_R1_20180508,20180508,134938,Free Whisking & Whisker Stim,Naive,NaN,Unknown,Unknown,Unknown
1,LB010,LB,LB010_S1_R3,Unknown,08.05.18,08.05.18,M,C57BL/6JRj,WT,25.02.2018,...,Unknown,LB010_S1_R3_20180508,20180508,152232,Free Whisking & Whisker Stim,Naive,NaN,Unknown,Unknown,Unknown


Converting data to NWB format for mouse: ['LB010_S1_R1_20180508', 'LB010_S1_R3_20180508']
📑 Created NWB files :
 
🔎 Validating NWB file before saving...
     o ✅ File data/LB010_S1_R1_20180508_134938.nwb is valid, no errors detected and saved successfully.
📃 Creating configs for NWB conversion :
     o ✅ File data/LB010_S1_R3_20180508_152232.nwb is valid, no errors detected and saved successfully.
**************************************************************************


In [ ]:
importlib.reload(converters.Initiation_nwb)
converters.Initiation_nwb.remove_nwb_files("data")